# Experiment 007: Apply fix_direction to Baseline

Strategy: Apply rotation optimization (fix_direction) to the baseline submission.
This optimizes the rotation angle for each N group to minimize the bounding box.

Key insight: This doesn't change overlaps - it just rotates the entire configuration.
So if the baseline passes Kaggle, the rotated version should also pass.

In [ ]:
import sys
sys.path.insert(0, '/home/code')

import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
import json
import shutil

getcontext().prec = 30

print("Libraries loaded successfully!")

In [ ]:
# Load the baseline submission
baseline_path = '/home/code/experiments/000_baseline/submission.csv'
baseline_df = pd.read_csv(baseline_path)
print(f"Baseline loaded: {baseline_df.shape}")
print(baseline_df.head())

In [ ]:
# Define the Christmas tree polygon (same as in utils.py)
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(0), float(tip_y)),
            (float(top_w / 2), float(tier_1_y)),
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),
            (float(trunk_w / 2), float(base_y)),
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),
            (float(-mid_w / 4), float(tier_2_y)),
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return s[1:]
    return str(s)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

print("ChristmasTree class defined.")

In [ ]:
# Implement fix_direction from bbox3 runner kernel
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees, angle_max=89.999, epsilon=1e-7):
    """Find optimal rotation angle to minimize bounding box."""
    # Get all polygon points
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    # Get convex hull points (more efficient)
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np
    
    # Calculate initial side
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Find optimal rotation angle
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, angle_max),
        method="bounded",
    )
    
    if initial_side - res.fun > epsilon:
        return res.x, res.fun, initial_side
    return 0.0, initial_side, initial_side

print("fix_direction functions defined.")

In [ ]:
# Test on a single N value
n = 10
trees = load_trees_for_n(baseline_df, n)
print(f"N={n}: {len(trees)} trees")

best_angle, best_side, initial_side = optimize_rotation(trees)
print(f"Initial side: {initial_side:.6f}")
print(f"Best angle: {best_angle:.4f} degrees")
print(f"Best side: {best_side:.6f}")
print(f"Improvement: {initial_side - best_side:.6f}")

In [ ]:
# Apply fix_direction to all N values
print("Applying fix_direction to all N values...")
print()

improvements = []
total_improvement = 0

for n in range(1, 201):
    trees = load_trees_for_n(baseline_df, n)
    if len(trees) != n:
        print(f"Warning: N={n} has {len(trees)} trees")
        continue
    
    best_angle, best_side, initial_side = optimize_rotation(trees)
    improvement = initial_side - best_side
    
    if improvement > 1e-7:
        # Calculate score improvement
        old_score = (initial_side ** 2) / n
        new_score = (best_side ** 2) / n
        score_improvement = old_score - new_score
        total_improvement += score_improvement
        improvements.append((n, best_angle, improvement, score_improvement))

print(f"\nFound {len(improvements)} N values with potential improvement")
print(f"Total score improvement: {total_improvement:.6f}")

# Show top improvements
improvements.sort(key=lambda x: -x[3])  # Sort by score improvement
print("\nTop 10 improvements:")
for n, angle, side_imp, score_imp in improvements[:10]:
    print(f"  N={n:3d}: angle={angle:.2f}°, side_imp={side_imp:.6f}, score_imp={score_imp:.6f}")

In [ ]:
# The fix_direction optimization finds the optimal rotation angle,
# but applying it requires rotating all tree positions and angles.
# This is complex and may introduce numerical precision issues.

# Let's check if the baseline already has optimal rotations
# by comparing with the theoretical improvement.

print("Analysis: The baseline may already have fix_direction applied.")
print(f"Total potential improvement: {total_improvement:.6f}")
print()

if total_improvement < 0.001:
    print("The baseline already has near-optimal rotations.")
    print("fix_direction provides negligible improvement.")
    print("\nUsing baseline as-is.")
    
    # Copy baseline to submission folder
    shutil.copy(baseline_path, '/home/code/experiments/007_fix_direction/submission.csv')
    shutil.copy(baseline_path, '/home/submission/submission.csv')
    
    # Save metrics
    metrics = {
        'cv_score': 70.676102,
        'baseline_score': 70.676102,
        'fix_direction_improvement': total_improvement,
        'notes': 'Baseline already has near-optimal rotations. fix_direction provides negligible improvement.'
    }
    with open('/home/code/experiments/007_fix_direction/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"\nMetrics saved: {metrics}")
else:
    print(f"Potential improvement of {total_improvement:.6f} points found!")
    print("Need to implement rotation application...")

In [ ]:
# Summary
print("=" * 60)
print("EXPERIMENT 007: FIX_DIRECTION SUMMARY")
print("=" * 60)
print(f"Baseline score: 70.676102")
print(f"Potential improvement from fix_direction: {total_improvement:.6f}")
print(f"N values with improvement: {len(improvements)}")
print(f"\nConclusion: {'Negligible improvement' if total_improvement < 0.001 else 'Significant improvement possible'}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {70.676102 - 68.888293:.6f} ({(70.676102 - 68.888293) / 68.888293 * 100:.2f}%)")
print("=" * 60)